# Prediction Model for BNPL Revenue

In [23]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import pandas as pd

from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import VectorAssembler 
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import *

In [24]:
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "2g")
    .config("spark.executer.memory", "4g")
    .getOrCreate()
)

## Data Preprocessing

In [26]:
full = spark.read.parquet('../data/curated/train_data/')
full = full.withColumnRenamed('y_total_num_consumer', 'next_total_num_consumer').withColumnRenamed('y_total_revenue', 'next_total_revenue')\
    .withColumnRenamed('y_total_num_transaction', 'next_total_num_transaction')
full.printSchema()

root
 |-- merchant_abn: long (nullable = true)
 |-- total_num_consumer: long (nullable = true)
 |-- avg_dollar_value: double (nullable = true)
 |-- total_num_transaction: long (nullable = true)
 |-- mean_income: double (nullable = true)
 |-- revenue_level: string (nullable = true)
 |-- total_revenue: double (nullable = true)
 |-- total_num_postcode: long (nullable = true)
 |-- tag: string (nullable = true)
 |-- next_total_num_consumer: long (nullable = true)
 |-- next_total_revenue: double (nullable = true)
 |-- next_total_num_transaction: long (nullable = true)



### Clustering

In [11]:
tag_mean = full.groupBy('tag')\
      .agg(
         F.mean("total_revenue").alias("mean_revenue_of_tags")
      )

In [12]:
tag_mean = tag_mean.toPandas()

In [13]:
from sklearn.cluster import KMeans
import numpy as np

kmeans = KMeans(n_clusters=4, random_state=0).fit(np.array(tag_mean['mean_revenue_of_tags']).reshape(-1, 1))
kmeans.labels_

array([2, 0, 0, 1, 2, 0, 3, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 2, 2, 1, 1, 2,
       1, 0], dtype=int32)

In [14]:
tag_mean['tag_labels'] = kmeans.labels_
tag_mean

,tag,mean_revenue_of_tags,tag_labels
0,jewelry,301542.063851,2
1,cable,545919.891131,0
2,watch,565603.630545,0
3,garden supply,732549.584612,1
4,antique,431668.237726,2
5,shoe,534436.300789,0
6,tent,995172.851759,3
7,stationery,518011.067448,0
8,artist supply,719151.356147,1
9,florists,637021.720045,1


In [15]:
tag_mean_sdf = spark.createDataFrame(tag_mean[['tag', 'tag_labels']])
tag_mean_sdf

tag,tag_labels
jewelry,2
cable,0
watch,0
garden supply,1
antique,2
shoe,0
tent,3
stationery,0
artist supply,1
florists,1


In [16]:
full = full.join(tag_mean_sdf, ["tag"], how="left") 
# use left join here since if no historical data is provided, we cannot predict the future value of a merchant
full

tag,merchant_abn,total_num_consumer,avg_dollar_value,total_num_transaction,mean_income,revenue_level,total_revenue,total_num_postcode,next_total_num_consumer,next_total_revenue,next_total_num_transaction,tag_labels
cable,10142254217,731,41.11965938159281,750,62693.954666666665,b,130143.7154723053,642,918,154467.92091896126,925,0
watch,10187291046,87,111.08408713922161,87,61060.0459770115,b,31795.597893195012,87,99,41683.211213258364,100,0
watch,10264435225,1238,114.10783402533234,1272,62006.311320754714,c,346896.95929006615,1018,1519,435003.67956298945,1566,0
artist supply,10385011947,6,526.8134669489664,6,55291.166666666664,b,10019.992382525408,6,9,18926.917288619916,9,1
artist supply,10463252268,22,464.0964976850653,22,60070.77272727273,a,67488.91405656068,22,26,78474.65405470507,26,1
artist supply,10487253336,629,449.0344083022495,635,63669.83464566929,b,1117736.4709001747,587,715,1206457.0200268542,733,1
florists,10385163239,17,350.08667638343934,17,58553.117647058825,a,39339.24061981451,16,22,52838.18504617855,22,1
music,10192359162,107,451.1432080236008,107,65637.3831775701,a,305563.802543579,107,107,290071.0062352741,107,0
music,10364012396,4,276.0868936989199,4,81123.75,b,4008.7818228908677,4,16,19636.79081402693,16,0
books,10530696903,174,429.6262128784364,175,61263.58857142857,a,485692.43652713683,167,226,606552.0421588303,226,1


## Model for BNPL Revenue
Features:
    total number of consumer, average dollar value, total number of transaction, mean income, total number of postcode, tags

label:
    next year revenue

In [27]:
# drop the columns not needed
revenue_df = full.drop('merchant_abn', 'revenue_level', 'total_revenue', 'next_total_num_consumer', 'next_total_num_transaction')
revenue_df

total_num_consumer,avg_dollar_value,total_num_transaction,mean_income,total_num_postcode,tag,next_total_revenue
808,209.02516040076958,823,62506.642770352366,719,furniture,38940.2914088986
731,41.11965938159281,750,62693.954666666665,642,cable,154467.92091896126
87,111.08408713922161,87,61060.0459770115,87,watch,41683.211213258364
107,451.1432080236008,107,65637.3831775701,107,music,290071.0062352741
2244,39.20921658203575,2370,62122.52278481013,1619,gift,666766.4067054291
218,389.5552654520502,218,63146.619266055044,211,computer,378005.14673149673
1238,114.10783402533234,1272,62006.311320754714,1018,watch,435003.67956298945
130,308.3515003987336,130,63676.28461538462,127,computer,273146.7707118801
2475,128.98933167858436,2621,62792.286913391836,1730,furniture,2765582.9188776133
238,378.0169713940927,238,61579.6512605042,227,computer,654693.9778432944


### Indexing and One-hot Encoding

In [18]:
# change tags into numeric feature by one hot encoding
indexer = StringIndexer(inputCol="tag", outputCol="tagIndex")
revenue_df = indexer.fit(revenue_df).transform(revenue_df)
ohe = OneHotEncoder(inputCol="tagIndex", outputCol="tagOHE")
revenue_df = ohe.fit(revenue_df).transform(revenue_df)
revenue_df

tag,total_num_consumer,avg_dollar_value,total_num_transaction,mean_income,total_num_postcode,next_total_revenue,tag_labels,tagIndex,tagOHE
jewelry,8,10439.401811028416,8,61840.875,8,null,2,23.0,"(23,[],[])"
jewelry,12,4209.2389409174475,12,63396.166666666664,12,3364.4301006651253,2,23.0,"(23,[],[])"
jewelry,1,6987.246435378608,1,48235.0,1,null,2,23.0,"(23,[],[])"
jewelry,2,1396.3251261623384,2,54250.5,2,null,2,23.0,"(23,[],[])"
jewelry,1,4798.332815388768,1,80991.0,1,null,2,23.0,"(23,[],[])"
jewelry,33,9848.725593936158,33,60891.90909090909,33,3940.16359943221,2,23.0,"(23,[],[])"
jewelry,29,14897.926207832394,29,62719.724137931036,28,null,2,23.0,"(23,[],[])"
jewelry,3,3622.5670910222148,3,66842.66666666667,3,null,2,23.0,"(23,[],[])"
jewelry,1,16658.482849584656,1,47353.0,1,null,2,23.0,"(23,[],[])"
jewelry,1,19486.763586439236,1,70738.0,1,null,2,23.0,"(23,[],[])"


In [19]:
import six
for i in revenue_df.columns[:-1]:
    if not( isinstance(revenue_df.select(i).take(1)[0][0], six.string_types)):
        print( "Correlation to next_total_revenue for ", i, revenue_df.stat.corr('next_total_revenue',i))

Correlation to next_total_revenue for  total_num_consumer 0.7410130770929826
Correlation to next_total_revenue for  avg_dollar_value -0.09722654487146351
Correlation to next_total_revenue for  total_num_transaction 0.6509065671273094
Correlation to next_total_revenue for  mean_income 0.014700804846086562
Correlation to next_total_revenue for  total_num_postcode 0.6656216278730779
Correlation to next_total_revenue for  next_total_revenue 1.0
Correlation to next_total_revenue for  tag_labels -0.028419755177593445
Correlation to next_total_revenue for  tagIndex -0.10497611182613942


### Vectorization

In [20]:
features = ['total_num_consumer', 'avg_dollar_value', 'total_num_transaction', 'mean_income', 'total_num_postcode', 'tagOHE']
assembler = VectorAssembler(inputCols=features, outputCol='features')
final_revenue_df = assembler.transform(revenue_df)
final_revenue_df = final_revenue_df.select('features','next_total_revenue')

### Model fitting

In [21]:
# missing values will not be included
train_df, test_df = (final_revenue_df.drop('tag', 'tagIndex').filter(F.col('next_total_revenue').isNotNull())).randomSplit([0.7, 0.3])

In [84]:
train_df, test_df = (final_revenue_df.filter((F.col('next_total_revenue').isNotNull())&(F.col('tag_labels')==1))).drop('tag', 'tagIndex', 'tag_labels').randomSplit([0.7, 0.3])

In [22]:
train_df.count(), test_df.count()

(2586, 1196)

#### Linear Regression

In [70]:
lr = LinearRegression(labelCol='next_total_revenue', maxIter=10, regParam=0.3, elasticNetParam=0.8)
fitted_model = lr.fit(train_df)
fitted_model.setFeaturesCol("features")
fitted_model.setPredictionCol("prediction")

LinearRegressionModel: uid=LinearRegression_6e451e88950d, numFeatures=28

In [71]:
print("Coefficients: %s" % str(fitted_model.coefficients))
print("Intercept: %s" % str(fitted_model.intercept))

Coefficients: [147.97843327117232,0.0,85.32570319599924,-0.18986513944769598,742.9054500956163,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3877.6635050366026,26675.10544928082,36123.76329516014,-40682.64170626974]
Intercept: 5998.93209095228


In [72]:
trainingSummary = fitted_model.summary
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
trainingSummary.residuals.show()
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

numIterations: 10
objectiveHistory: [0.5, 0.4385975131930148, 0.28474438258461315, 0.2438504274460106, 0.2320701436019277, 0.22791134986155795, 0.22516858238985782, 0.2223296969928053, 0.2185053253088702, 0.21622820881221053, 0.21353154419677323]
+------------------+
|         residuals|
+------------------+
| 7432.119075475288|
|2346.3565332611465|
|  8554.88947340806|
|  34323.6694849341|
|12593.180083608859|
|3102.1682635621364|
|15897.226152553725|
| 2322.704138401477|
| 7172.715790443264|
|2026.8890543954844|
|7144.1179169566485|
|20944.705869157704|
|281.43900160423595|
| 6544.546269235913|
| -512.605162724487|
| 9765.170990116087|
|-559.7171232905239|
|7021.5244212037505|
|18164.623668186414|
|-34480.81848949313|
+------------------+
only showing top 20 rows

RMSE: 218965.638903
r2: 0.572938


In [73]:
train_df.describe().show()

+-------+------------------+
|summary|next_total_revenue|
+-------+------------------+
|  count|               260|
|   mean|115747.37453880043|
| stddev| 335712.4942268983|
|    min|146.56267111256705|
|    max| 2578899.989549712|
+-------+------------------+



##### Linear Regression Evaluation

In [74]:
lr_predictions = fitted_model.transform(test_df)
lr_predictions.select("prediction","next_total_revenue","features").show(10)
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="next_total_revenue",metricName="r2")
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(lr_predictions))

+-------------------+------------------+--------------------+
|         prediction|next_total_revenue|            features|
+-------------------+------------------+--------------------+
|-2507.8625773395534| 700.0913339069108|(28,[0,1,2,3,4],[...|
| -3763.531377081882| 5551.664760915629|(28,[0,1,2,3,4],[...|
| -3820.870649195086|15361.340843110367|(28,[0,1,2,3,4],[...|
| -1093.123507037396| 6330.006413055507|(28,[0,1,2,3,4],[...|
| 439.78965489998154|12730.592472437624|(28,[0,1,2,3,4],[...|
|-1366.2075515265024| 2973.525203961843|(28,[0,1,2,3,4],[...|
| 1848.6737562512335| 7714.789567461943|(28,[0,1,2,3,4],[...|
| 3023.4144696979356|3284.6497179037897|(28,[0,1,2,3,4],[...|
|   5876.74802949784| 71807.61440922701|(28,[0,1,2,3,4],[...|
|  5676.725105089693|3364.4301006651253|(28,[0,1,2,3,4],[...|
+-------------------+------------------+--------------------+
only showing top 10 rows

R Squared (R2) on test data = 0.613892


In [106]:
from pyspark.ml.regression import GeneralizedLinearRegression
glr = GeneralizedLinearRegression(labelCol='next_total_revenue', family="gamma", link="inverse", maxIter=10, regParam=0.3)
model = glr.fit(train_df)
summary = model.summary

In [107]:
print("Coefficients: " + str(model.coefficients))
print("Intercept: " + str(model.intercept))

Coefficients: [-5.081981829145639e+35,-2.6870343238436396e+34,6.678906219871052e+34,-2.163592960903312e+33,-2.294703291151991e+34,-4.92718762286847e+36,8.400747166426864e+36,-5.605131552930673e+36,-1.7198318993286945e+36,3.2235886586410473e+36,-2.158475653750893e+35,-3.7379337761048877e+36,4.818852487444756e+36,3.9813588208546056e+36,4.080970583519597e+36,-4.676779563277705e+35,-3.0601522088596924e+36,-3.286057039141181e+36,-2.873629466789348e+36,-3.8704207786982195e+36,-1.757855944350198e+36,2.783813584487964e+36,-2.069366545011462e+36,6.517096251093483e+35,1.4196427137124834e+36,1.2734316058753377e+36,7.62234692965126e+35,8.931461421116118e+35]
Intercept: 9.011220949745152e+37


In [108]:
print("Coefficient Standard Errors: " + str(summary.coefficientStandardErrors))
print("T Values: " + str(summary.tValues))
print("P Values: " + str(summary.pValues))
print("Dispersion: " + str(summary.dispersion))
print("Null Deviance: " + str(summary.nullDeviance))
print("Residual Degree Of Freedom Null: " + str(summary.residualDegreeOfFreedomNull))
print("Deviance: " + str(summary.deviance))
print("Residual Degree Of Freedom: " + str(summary.residualDegreeOfFreedom))
print("AIC: " + str(summary.aic))
print("Deviance Residuals: ")
summary.residuals().show()

Coefficient Standard Errors: [1.0313239414198771e+34, 3.478628089243441e+33, 3.5683495996081674e+33, 1.0816705970704932e+33, 1.4503718984533777e+34, 5.539912749322146e+36, 5.893721277193464e+36, 5.844093458400502e+36, 5.876675126188564e+36, 5.87325712140477e+36, 5.848886994636668e+36, 5.868749562497159e+36, 5.855177161171895e+36, 5.877248737248757e+36, 5.896620810168444e+36, 5.903887371388085e+36, 5.905115307859655e+36, 5.904614198395587e+36, 5.937275150656821e+36, 5.911421045557285e+36, 5.938595972689419e+36, 5.938813105595627e+36, 5.954762089698656e+36, 5.961939652059689e+36, 6.033865684468139e+36, 6.07838891654262e+36, 6.110853304354673e+36, 6.151512087390794e+36, 6.8105032215591325e+37]
T Values: [-49.276290649754635, -7.724408171579033, 18.71707363147503, -2.000232757332045, -1.5821482018501438, -0.8893980547746626, 1.4253723193417152, -0.9591105263509541, -0.2926539007855835, 0.5488587664403202, -0.036904040986433476, -0.6369216706726183, 0.8230071191356194, 0.6774188057792411, 0

#### Random Forest Regressor

In [101]:
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

In [102]:
rf = RandomForestRegressor(numTrees = 10, maxDepth = 10, labelCol='next_total_revenue')

In [103]:
model = rf.fit(train_df)
result = model.transform(test_df)

In [104]:
rf_evaluator = RegressionEvaluator(labelCol='next_total_revenue', metricName="mae", predictionCol='prediction')
mae = rf_evaluator.evaluate(result)
rf_evaluator = RegressionEvaluator(labelCol='next_total_revenue', metricName="r2", predictionCol='prediction')
r2 = rf_evaluator.evaluate(result)
print('+++++++++++++++++++++++++++++++++++++++++++')
print(f'Using Categorical feature: {features}')
print('mae:{}'.format(mae))
print('r2: {}'.format(r2))

+++++++++++++++++++++++++++++++++++++++++++
Using Categorical feature: ['total_num_consumer', 'avg_dollar_value', 'total_num_transaction', 'mean_income', 'total_num_postcode', 'tagOHE']
mae:305911.1848229039
r2: 0.5990584247955485


In [105]:
result.select("prediction","next_total_revenue","features").show(20)

+------------------+------------------+--------------------+
|        prediction|next_total_revenue|            features|
+------------------+------------------+--------------------+
| 9853.891544733393| 5551.664760915629|(28,[0,1,2,3,4],[...|
| 9232.881639069303| 7714.789567461943|(28,[0,1,2,3,4],[...|
|40613.649820138045|39997.213284193116|(28,[0,1,2,3,4],[...|
|139927.58291571058|21404.354662000824|(28,[0,1,2,3,4],[...|
|27048.383025172265| 21214.30272082452|(28,[0,1,2,3,4,5]...|
|23631.234895597558| 38692.67439389365|(28,[0,1,2,3,4,5]...|
|27895.020971600858| 22074.99856066441|(28,[0,1,2,3,4,5]...|
|28025.083524159854| 75683.95506697436|(28,[0,1,2,3,4,5]...|
| 29028.92472391144|25826.219229145478|(28,[0,1,2,3,4,5]...|
| 38457.01274226319|28967.859022303237|(28,[0,1,2,3,4,5]...|
|59881.001715998085|18736.186917799685|(28,[0,1,2,3,4,5]...|
|58545.640659440716|22537.278176372223|(28,[0,1,2,3,4,5]...|
|65389.267248563505|32046.023857037744|(28,[0,1,2,3,4,5]...|
| 38314.06821998696| 263

#### Gradient Boosting Tree

In [19]:
from pyspark.ml.regression import GBTRegressor
gbt = GBTRegressor(featuresCol = 'features', labelCol = 'next_total_revenue', maxIter=10)
gbt_model = gbt.fit(train_df)
gbt_predictions = gbt_model.transform(test_df)
gbt_predictions.select('prediction', 'next_total_revenue', 'features').show(5)

+------------------+------------------+--------------------+
|        prediction|next_total_revenue|            features|
+------------------+------------------+--------------------+
|19566.260698153033| 700.0913339069108|(28,[0,1,2,3,4],[...|
| 46132.53320660531|20115.019410151996|(28,[0,1,2,3,4],[...|
| 46132.53320660531|25665.858312655597|(28,[0,1,2,3,4],[...|
|19566.260698153033|24217.897520994058|(28,[0,1,2,3,4,5]...|
|34625.277661725544| 7335.947872319652|(28,[0,1,2,3,4,5]...|
+------------------+------------------+--------------------+
only showing top 5 rows



In [20]:
gbt_evaluator = RegressionEvaluator(
    labelCol="next_total_revenue", predictionCol="prediction", metricName="rmse")
rmse = gbt_evaluator.evaluate(gbt_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 932610


#### Multilayer Perceptron Regressor

In [80]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import sklearn.metrics as metrics
from sklearn.model_selection import GridSearchCV

In [75]:
revenue_pd = revenue_df.toPandas().dropna()
revenue_pd

,total_num_consumer,avg_dollar_value,total_num_transaction,mean_income,total_num_postcode,tag,next_total_revenue
0,808,209.025160,823,62506.642770,719,furniture,3.894029e+04
1,731,41.119659,750,62693.954667,642,cable,1.544679e+05
2,87,111.084087,87,61060.045977,87,watch,4.168321e+04
3,107,451.143208,107,65637.383178,107,music,2.900710e+05
4,2244,39.209217,2370,62122.522785,1619,gift,6.667664e+05
...,...,...,...,...,...,...,...
3948,3920,30.917688,4285,62782.274912,2255,opticians,7.204531e+05
3949,33,319.209608,33,60450.212121,33,books,3.991901e+04
3950,5353,150.114508,6027,62576.340468,2607,shoe,6.974851e+06
3951,45,353.519570,45,61502.444444,45,motor,1.762053e+05


In [76]:
features_pd = ['total_num_consumer', 'avg_dollar_value', 'total_num_transaction', 'mean_income', 'total_num_postcode']
X = revenue_pd[features_pd]
y = revenue_pd['next_total_revenue'].to_numpy()

In [77]:
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=30034, test_size=0.3)

In [83]:
# scale train and test dataset in order to be standard normally distributed with zero mean
sc_X = StandardScaler()
X_trainscaled=sc_X.fit_transform(X_train)
X_testscaled=sc_X.transform(X_test)
X_trainscaled.shape, X_testscaled.shape

((2647, 5), (1135, 5))

##### Hyperparameters

In [85]:
mlp_reg = MLPRegressor(hidden_layer_sizes=(128,128,128),activation="relu" ,solver = 'adam', random_state=30034, max_iter=20000)\
    .fit(X_trainscaled, y_train)
y_pred=mlp_reg.predict(X_testscaled)
print("The Score with ", (metrics.r2_score(y_pred, y_test)))

In [88]:
# using grid search for best parameter combinations
param_grid = {
    'hidden_layer_sizes': [(150,100,50), (120,80,40), (100,50,30)],
    'activation': ['identity', 'logistic', 'tanh', 'relu'],
    'solver': ['sgd', 'adam', 'lbfgs'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}

grid = GridSearchCV(mlp_reg, param_grid, n_jobs= -1, cv=5)
grid.fit(X_trainscaled, y_train)
print(grid.best_params_)

/home/jinxuan/.local/lib/python3.8/site-packages/sklearn/utils/extmath.py:152: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/home/jinxuan/.local/lib/python3.8/site-packages/sklearn/utils/extmath.py:152: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/jinxuan/.local/lib/python3.8/site-packages/sklearn/utils/extmath.py:152: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/home/jinxuan/.local/lib/python3.8/site-packages/sklearn/utils/extmath.py:152: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/jinxuan/.local/lib/python3.8/site-packages/sklearn/utils/extmath.py:152: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/home/jinxuan/.local/lib/python3.8/site-packages/sklearn/utils/extmath.py:152: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/jinxuan/.local/lib/python3.8/site-packages/sklearn/utils/extmath.py:152: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/home

KeyboardInterrupt: 

In [ ]:
grid_predictions = grid.predict(X_testscaled)
grid_result = pd.DataFrame({'Actual': y_test, 'Predicted': grid_predictions})
grid_result.head()

For solver: 

The default solver ‘adam’ works pretty well on relatively large datasets (with thousands of training samples or more) in terms of both training time and validation score. For small datasets, however, ‘lbfgs’ can converge faster and perform better.

For activition:

‘identity’, no-op activation, useful to implement linear bottleneck, returns f(x) = x

‘logistic’, the logistic sigmoid function, returns f(x) = 1 / (1 + exp(-x)).

‘tanh’, the hyperbolic tan function, returns f(x) = tanh(x).

‘relu’, the rectified linear unit function, returns f(x) = max(0, x)

(from Sklearn)

##### Evaluation

In [ ]:
df_result = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
df_result.head()

In [ ]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

In [ ]:
print('Mean Absolute Error for grid search:', metrics.mean_absolute_error(y_test, grid_predictions))  
print('Mean Squared Error for grid search:', metrics.mean_squared_error(y_test, grid_predictions))  
print('Root Mean Squared Error for grid search:', np.sqrt(metrics.mean_squared_error(y_test, grid_predictions)))
